## Orchestration with Step Functions

In this example the AWS Step Function service uses a state machines to orchestrate the execution of AWS Lambda functions. A client sends a request to an Amazon API Gateway which in turn passes that request to the Step Function service. Included in the request sent to the Step Function service is the body of information to be processed and the state machine to be executed. 

The state machine begins by calling a Lambda function which writes input to a DynamoDB table. If that operation is successful the state machine calls the next function which writes the input as an object to an Amazon S3.

![architecture](../images/architecture_3.png "Architecture")

### Demonstration Steps
1.  The first command posts data to the API Gateway, which in turn passes the request body and state machine name to the AWS Step Function service. Use the Step Function console to view the successful execution of the state machine.


2.	The second script is the json formatter.


3.	The third script sends a GET request to the API endpoint which will call a Lambda function that executes a Get Item operation against DynamoDB.


4.	The next script sends a GET request to the API endpoint which will call a Lambda function that generates a pre-signed URL for an object in S3.


5.	After you have successfully executed the state machine, modify the state machine and introduce a **SNS Publish** step to send an email notification when all operations succeed. 


6.	Test the modified state machine by placing another order. You should receive a single email.


7.	The final script returns the previous order was placed.

**Final State Machine**
![architecture](../images/architecture_2.png "Architecture")

**Note:** Make sure you set **gwid** to your gateway id using - gwid = '...'

The first command posts data to the API Gateway, which in turn passes the request body and state machine name to the AWS Step Function service.

**Post order**

In [ ]:
import boto3, requests, datetime
from random import randrange

#Set gateway id
gwid = '...'

city_list=["Adelaide","Brisbane","Canberra","Darwin","Geelong","Gold Coast","Hobart","Melbourne","Perth","Sydney","Wollongong"]
coffeetype_list=["Short Black","Flat White","Latte","Long Black"]
coffeesize_list=[{"size":"Small","price":"3.5"},{"size":"Medium","price":"4.0"},{"size":"Large","price":"4.5"},{"size":"x-Large","price":"5.0"}]

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

for i in range(5):
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #Original value-----------
    coffeesize = randrange(4)
    #-------------------------

    quantity = randrange(10)+1
    city = city_list[randrange(11)]
    eventtype="new_order"
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,
            'details':{
                'coffeetype': coffeetype_list[randrange(4)],
                'coffeesize': coffeesize_list[coffeesize]["size"],
                'unitprice': coffeesize_list[coffeesize]["price"],
                'quantity': quantity
            },
            'eventtype':[eventtype]
        }
    })

print(response)

**json formatter** - Run the following script to create a class which will be used to render json objects in a readable format.

In [ ]:
import json, uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

**Get Order** from Amazon DynamoDB using the primary key (accountid & vendorid)

In [ ]:
url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

response_get = requests.get(url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())

**Get pre-signed url from S3**, use the url to retrieve the object.

In [ ]:
#Get Script

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/invoice')

now = datetime.datetime.now()
path = str(now.year) + '/' + str(now.month) + '/'

response_get = requests.get(url, params={'objectKey':path + 'a0'})

print(response_get.json())

**SNS Notification**. Using the **Step Functions Workflow Studio** edit the state machine and insert a **SNS Publish** step between **task_put_object** and **Job Succeeded**. Configure the step with the following settings.

Topic ARN: ```accountid:step_function_orhestration_topic```

**SNS Subscription**. You will also need to setup and confirm a SNS Subscription.

Protocol: ```Email```

Endpoint: ```Your email```


In [ ]:
url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

accountid = 'a100'
vendorid = 'v100'
orderdate = str(datetime.datetime.now())

#Original value-----------
coffeesize = randrange(4)
#-------------------------

quantity = randrange(10)+1
city = city_list[randrange(11)]
eventtype="new_order"
response = requests.post(url,json={'order':{
        'accountid': accountid,
        'orderdate':orderdate,
        'vendorid': vendorid,
        'city':city,
        'details':{
            'coffeetype': coffeetype_list[randrange(4)],
            'coffeesize': coffeesize_list[coffeesize]["size"],
            'unitprice': coffeesize_list[coffeesize]["price"],
            'quantity': quantity
        },
        'eventtype':[eventtype]
    }
})
print(response)

**Get Order** from Amazon DynamoDB using the primary key (accountid & vendorid)

In [ ]:
url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

response_get = requests.get(url, params={'accountid':'a100','vendorid':'v100'})

RenderJSON(response_get.json())